In [35]:
import numpy as np
from nltk.corpus import brown, stopwords
from nltk.cluster.util import cosine_distance
import tensorflow as tf
import pandas as pd
from collections import defaultdict
import json
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk import word_tokenize, tokenize
from nltk.stem import WordNetLemmatizer 
import io
from operator import itemgetter 

In [2]:
def sentence_similarity(s1, s2, stopwords=None):
    if stopwords is None:
        stopwords = []
 
    sent1 = [w.lower() for w in s1.split()]
    sent2 = [w.lower() for w in s2.split()]
 
    all_words = list(set(sent1 + sent2))
 
    vector1 = [0] * len(all_words)
    vector2 = [0] * len(all_words)
 
    # build the vector for the first sentence
    for w in sent1:
        if w in stopwords:
            continue
        vector1[all_words.index(w)] += 1
 
    # build the vector for the second sentence
    for w in sent2:
        if w in stopwords:
            continue
        vector2[all_words.index(w)] += 1
 
    return 1 - cosine_distance(vector1, vector2)

In [3]:

# One out of 5 words differ => 0.8 similarity
print(sentence_similarity("This is a good sentence", "This is a bad sentence"))

stop_words = stopwords.words('english')

0.7999999999999998


In [31]:

def build_similarity_matrix(sentences, stopwords=None):
    # Create an empty similarity matrix
    S = np.zeros((len(sentences), len(sentences)))
 
 
    for idx1 in range(len(sentences)):
        for idx2 in range(len(sentences)):
            if idx1 == idx2:
                continue
       
            S[idx1][idx2] = sentence_similarity(sentences[idx1], sentences[idx2], stop_words)
 
    # normalize the matrix row-wise
    for idx in range(len(S)):
        if S[idx].sum() == 0:
            continue
        else:
            S[idx] /= S[idx].sum()
 
    return S

In [5]:
def pagerank(A, eps=0.0001, d=0.85):
    P = np.ones(len(A)) / len(A)
    while True:
        new_P = np.ones(len(A)) * (1 - d) / len(A) + d * A.T.dot(P)
        delta = abs(new_P - P).sum()
        if delta <= eps:
            return new_P
        P = new_P


In [38]:
def textrank(sentences, top_n=5, stopwords=None):
    """
    sentences = a list of sentences [[w11, w12, ...], [w21, w22, ...], ...]
    top_n = how may sentences the summary should contain
    stopwords = a list of stopwords
    """
    S = build_similarity_matrix(sentences, stop_words) 

    sentence_ranks = pagerank(S)
 
    # Sort the sentence ranks
    ranked_sentence_indexes = [item[0] for item in sorted(enumerate(sentence_ranks), key=lambda item: -item[1])]
    selected_sentences = sorted(ranked_sentence_indexes[:top_n])
    summary = itemgetter(*selected_sentences)(sentences)
    return summary
 

In [7]:
path = '/Users/alexhe/Desktop/cs221project/reviews_Automotive_5.json'
data = []
with open(path) as f:
    for line in f:
        data.append(json.loads(line))


In [8]:

X=[] #reviews
Y=[] #ratings
asin_dict = defaultdict(list)
review_count = 0

# Assigns X as a list of reviews, Y as a list of star ratings, and asin_dict as a dictionary with keys asin_pos and asin_neg
# where asin_pos contains all reviews associated with that asin with a star-rating of 4 or 5, and asin_neg has all reviews
# for that asin with a star rating of 1, 2, or 3
for i in data:
    review_count += 1
    X.append(i['reviewText'])
    Y.append(i['overall'])
    if i['overall'] >= 4:
        asin_dict[i['asin']+"_pos"].append(i['reviewText'])
    else:
        asin_dict[i['asin']+"_neg"].append(i['reviewText'])

# Merges the list of reviews for each asin_pos and asin_neg into a single string
for j in asin_dict.keys():
    asin_dict[j] = " ".join(asin_dict[j])

In [9]:
print review_count
print X[1]
print Y[1]


20473
These long cables work fine for my truck, but the quality seems a little on the shabby side. For the money I was not expecting 200 dollar snap-on jumper cables but these seem more like what you would see at a chinese knock off shop like harbor freight for 30 bucks.
4.0


In [22]:
dict = asin_dict.keys()
print dict[0]
print asin_dict[dict[0]]

B000BPZ824_pos
[u'I wanted a superior quality...Royal Purple is the one!', u'If you have an oil requirement - get the very best and relax.', u'I have all Husquvarna 10 year old weed eater, 25 year old chain saw and 2 year old blower.', u'Mix this with 100 percent gas a feel the difference This stuff is amazing!', u'Oils that I used in the past, cannot compete with Royal Purple two cycle oil.', u'My mower no longer vibrates, sputters, or is it loud.', u'It runs very very smooth and quiet for a two cycle engine.', u"i'm not a pro, but i can tell the engine is smooter, no smoke and it is still running.", u'no complain from here.', u'Not only is this a better deal than the one bottle applications, it is an excellent quality 2-cycle oil.', u'After one tankful I could definitely tell the difference.', u'I would recommend this to anyone.', u'I LOVE the Royal Purple oils and have them in my car and scooter.', u'It makes them run so smoothly.']


In [11]:
for key in asin_dict.keys():
    asin_dict[key] = tokenize.sent_tokenize(asin_dict[key])


KeyboardInterrupt: 

In [ ]:
holder = True
i = 0
count

while holder:

    if len(asin_dict[dict[i]]) >= 30:
        count += 1
        print dict[i]
        print asin_dict[dict[i]]
        for idx, sentence in enumerate(textrank(asin_dict[dict[i]], stopwords=stop_words)):
            print("%s. %s" % ((idx + 1), ' '.join(sentence)))
    if count == 5:
        break
    i += 1

B000EVWDU0_pos
[u'I Bought this for 2 reasons.', u'One as a response to my car battery acting weird and another to keep an eye on my boat batteries.', u'It is very accurate, no problems at all.', u'I recently purchased a Schumacher underhood 1.5 amp Trickle Charger, this is a great product to work with a unit like that.', u'You can monitor the charging status of the battery as well as the alternator charging the system with the engine running.', u'I verified the accuracy with both my fluke and extech meters.', u'As winter approaches in Illinois it is important to insure my vehicles are at peak performance all winter.', u'I have one vehicle that sits outside in the cold with an elect.', u'battery blanket, block heater, trickle charger and now I can insure all systems are "go" with this product plugged in.', u'I am confident I will be able to easily start my truck no matter what winter throws at me.This seems well built and I was so impressed initially that I picked up two more as x-mas 

1. I   h a v e   a n   o l d e r   b a t t e r y   a n d   w o u l d   r e c o m m e n d   t h i s   t o   a n y   o n e   w i t h   f r e q u e n t   b a t t e r y   t r o u b l e s .
2. M o s t   a l l   m u l t i m e t e r s   a l s o   r e q u i r e   a   b a t t e r y   i n   t h e m   s o   e v e n   i f   y o u   s t o r e   o n e   o f   t h o s e   i n   y o u r   v e h i c l e ,   t h e   c h a n c e s   a r e   h i g h   t h a t   w h e n   y o u   n e e d   t o   u s e   i t ,   t h e   i n t e r n a l   b a t t e r y   w i l l   b e   d e a d . T h i s   E q u u s   u n i t   h o w e v e r   r e q u i r e s   n o   i n t e r n a l   b a t t e r y   ( a t   l e a s t   I   d o n ' t   t h i n k   i t   h a s   o n e ) ,   s o   y o u   c a n   j u s t   t h r o w   i t   i n   y o u r   g l o v e b o x   i n d e f i n i t e l y . C a r   b a t t e r i e s   t y p i c a l l y   l a s t   a r o u n d   5   y e a r s   s o   a f t e r   a b o u t   3 . 5   y e a r s   i s   w 

1. A l t h o u g h   y o u   c a n   r e p a i r   a   t i r e   o n   t h e   c a r ,   y o u   n e e d   s o   m u c h   l e v e r a g e   t o   d r i v e   t h e   r e a m e r ,   ( a n d   t h e   p l u g   n e e d l e )   t h r o u g h   t h e   t r e a d   a n d   t r e a d   b e l t s ,   i t   w i l l   b e   d i f f i c u l t .
2. I   r e c e i v e d   t h i s   t i r e   r e p a i r   k i t   a s   a   g i f t   f o r   m y   b i r t h d a y .
3. T h i s   s e t   c a n   r e p a i r   a   t i r e   i n   n o   t i m e   a n d   b e   p e r m a n e n t .
4. A b s o l u t e l y   t h e   b e s t ,   a n d   h i g h l y   r e c o m m e n d e d . . . :   )   H a v e n ' t   h a d   a   c h a n c e   t o   u s e   t h e s e   y e t   b u t   t h e y   l o o k   v e r y   w e l l   m a d e ,   m u c h   b e t t e r   t h a n   y o u r   t y p i c a l   a u t o m o t i v e   s t o r e   c r a p   I   h a v e   p u r c h a s e d   t h o s e   c h e a p   k i t s   f o r   t i r e   

1. L e x o l   l e a t h e r   c o n d i t i o n e r   d i d   a   f i n e   j o b .
2. I   u s e   t h e   l e a t h e r   c o n d i t i o n e r   o n   t h e   l e a t h e r   i n   m y   a l m o s t   4   y e a r   o l d   c a r ,   a s   w e l l   a s   a n   o l d   c o m f y   l e a t h e r   a r m   c h a i r .
3. I   u s e   t h i s   o n   m y   l e a t h e r   s e a t s   a f t e r   c l e a n i n g   t h e m   w i t h   t h e   L e x o l   p H   l e a t h e r   c l e a n e r .
4. I   j u s t   b o u g h t   t h i s   f o r   m y   l a m b s k i n   l e a t h e r   j a c k e t   a n d   m y   l e a t h e r   g l o v e s .
5. I   u s e   L e x o l   l e a t h e r   c o n d i t i o n e r   o n   t h e   l e a t h e r   s e a t s   i n   t h e   c a r   a n d   l e a t h e r   f u r n i t u r e .
B0002UQATS_pos
[u'Like the shine so far, will see how long it lasts, one week so far and water is still beading off the car Product is easy on/off.', u'Provides a decent length of prote

1. I ' l l   w a x   w i t h   t h i s   s t u f f   a b o u t   o n c e   a   m o n t h   a n d   e v e r y   w e e k e n d   w h e n   I   d o n ' t   u s e   t h i s   w a x   I   u s e   t h e M e g u i a r ' s   U l t i m a t e   Q u i k   W a x   G o e s   o n   S U P E R   e a s y .
2. v e r y   e x p e n s i v e   a n d   l i k e   a l l   c o a t i n g s   y o u   h a v e   t o   l o v e   t o   d o   t h i s   s t u f f   t o   y o u r   c a r   -   s t i l l ,   t h i s   i s   a   q u a l i t y   p r o d u c t   f r o m   a   t r u s t e d   n a m e   a l t h o u g h   i t   d o e s   r e q u i r e   r e g u l a r   r e a p p l i c a t i o n   m o r e   o f t e n   t h a n   i t   c l a i m s   t o   w h e t h e r   o r   n o t   y o u r   c a r   g o e s   t h r o u g h   a   c a r   w a s h   o r   i s   w a s h e d   b y   h a n d   T h e   b e n e f i t s   o f   t h i s   w a x   a r e   t h a t   i t   g o e s   o n   a n d   c o m e s   o f f   l i k e   b u t t e r 

...

1. I   o r i g i n a l l y   t h o u g h t   I   w o u l d   u s e   i t   t o   p o u r   g a s   i n t o   l a w n   c a r e   d e v i c e s   b u t   w h e n   i t   a r r i v e d   m y   w i f e   h a d   a   n e e d   f o r   a   f u n n e l   t o   u s e   i n   t h e   k i t c h e n .
2. P e r f e c t   f u n n e l   t o   p o u r   o i l   i n t o   y o u r   c r a n k c a s e . .
3. I   l o v e   t h i s   f u n n e l   m o r e   t h a n   o t h e r s   t h a t   I   h a v e   b e c a u s e   i t   i s   t h e   o n l y   o n e   t h a t   s t a y s   i n   w h i l e   I   a m   p o u r i n g .
4. T h e r e   a r e   n o   f r i l l s   h e r e . . .   y o u   p o u r   t h i n g s   t h r o u g h   t h i s   i n t o   o t h e r   t h i n g s . . .   e n d   o f   s t o r y . I f   t h a t ' s   w h a t   y o u ' r e   t r y i n g   t o   d o ,   I   h i g h l y   r e c o m m e n d   t h i s   o n e   I   g o t   s i c k   o f   u s i n g   m y   & # 3 4 ; C o m m o n & # 3 4 

1. I   h a v e   b e e n   u s i n g   t h i s   b r a n d   o f   w a x   f o r   a b o u t   3   y e a r s .
2. T h i s   c a r   w a x   i s   t h e   m o s t   d u r a b l e   I ' v e   u s e d ,   l e a v e s   a   g r e a t   s h i n e   a n d   r e a l l y   p r o t e c t s   a g a i n s t   r a i n   a n d   t h e   e n v i r o n m e n t .
3. R e a s o n   i s   h a v i n g   a   c l a y e d   a n d   p o l i s h e d   p a i n t   f i n i s h e d   w i l l   m a k e   s u r e   t h e   w a x   i s   w o r k i n g   o n   a   c l e a n   a n d   e v e n   s u r f a c e .
4. I   t h i n k   i t   l a s t s   l o n g e r   t h a n   t h e   t y p i c a l   w a x   a n d   e v e n   l o n g e r   t h a n   m o s t   s e a l a n t s .
5. T h e   4 7 6   f e a t u r e s   c a r n a u b a   w a x   A N D   n o r m a l   w a x   t o g e t h e r   w i t h   s o m e   p o l y m e r s   t o   m a l l   a l l   w o r k   t o g e t h e r .
B001DKTG0G_pos
[u"I replaced the manual transmissio

1. I   n e e d e d   a   g o o d   g r e a s e   g u n   w i t h   a   f l e x i b l e   h o s e   t o   p r o p e r l y   l u b e   m y   r i d i n g   m o w e r   a n d   t h i s   p r o d u c t   d o e s   a   g o o d   j o b .
2. O v e r a l l ,   I   w o u l d   r e c o m m e n d   t h i s   g u n   b a s e d   o n   t h e   t w o   b o t t l e s   o f   g r e a s e   I ' v e   u s e d   s o   f a r .
3. I t   i s   t h e   f i r s t   g r e a s e   g u n   t h a t   p u t   m o r e   g r e a s e   i n t o   t h e   j o i n t   t h a n   o n   m y   h a n d s .
4. T h e   p i s t o l   s t y l e   g r e a s e   g u n   i s   a n   a b s o l u t e   m u s t .
5. A d d i t i o n a l l y   t h e   q u a l i t y   o f   t h i s   g r e a s e   g u n   i s   b e t t e r   t h a n   a n y   I   h a v e   u s e d .
B0007O8S58_pos
[u'I installed these on my 2005 GMC Envoy XL with brush guard and they work and look great.', u'They really light up the offroad trails and remote roads at nigh

1. T h i s   P u t c o   9 0 0 0 9 - 6 0   L E D   T a i l g a t e   L i g h t   B a r   w i t h   T u r n   S i g n a l   i s   e a s y   t o   i n s t a l l   i f   y o u   h a v e   a   t o w   p a c k a g e   &   a   p l a c e   t o   p l u g   i t   i n .
2. I f   y o u   d o n ' t   h a v e   a   p l u g   i n   a n d   n o   a u t o m o t i v e   k n o w l e d g e ,   t h e   l i g h t   b a r   m a y   b e   a   l i t t l e   m o r e   d i f f i c u l t   t o   i n s t a l l . I   i n s t a l l e d   t h e   l i g h t   b a r   u s i n g   t h e   e n c l o s e d   2 - s i d e d   t a p e   ( d i d   n o t   w a n t   t o   d r i l l   h o l e s   i n t o   m y   t a i l g a t e ) .
3. S o   f a r ,   a f t e r   a p p r o x i m a t e l y   2 0 0   m i l e s ,   t h e   l i g h t   b a r   i s   s t i l l   s e c u r e l y   f a s t e n e d   t o   t h e   t a i l g a t e . T h e   L E D   l i g h t   i s   b r i g h t ,   a n d   i s   b r i g h t   e n o u g h   s o   t h a t

1. t h e s e   c a b l e s   a r e   m a d e   v e r y   n i c e , v e r y   s t r o n g   s h o u l d   b e   e a s y   t o   u s e   a n d   c h a r g e   a   c a r   w h e n   n e e d e d , i t   c o m e s   w i t h   a   n i c e   c l o t h   b a g   s o   y o u   c a n   s t o r e   t h e m   a w a y   w h e n   n o t   u s i n g   t h e m   T h e   1 6   f o o t   r e a c h   i s   a m a z i n g   a n d   a   m u s t   h a v e .
2. S o m e   j u m p e r   c a b l e s   a r e   t o o   s h o r t ,   b u t   t h e s e   a r e   p l e n t y   l o n g   f o r   e m e r g e n c y   u s e .
3. S o ,   i n s t e a d   o f   p u s h i n g   i t   o u t   o f   t h e   s p a c e ,   w e   w e r e   a b l e   t o   s t r e t c h   t h e   j u m p e r s   l o n g   e n o u g h   t o   r e a c h   t h e   b a t t e r y . E d i t   7 / 1 7 / 1 3 : S u m m a r y :   G r e a t   c a b l e s ,   f u n n y   p r o n g e d   t i p ,   t h r e w   a w a y   t h e   b a g   y e a r s   a g o . . . T